In [1]:
import pandas as pd

import statsmodels.api as sm
import pandas as pd
import datetime
from pathlib import Path

pd.set_option("display.max_columns", None)
# !pip install statsmodels


# %config InlineBackend.figure_format = "svg"
# %config InlineBackend.print_figure_kwargs = {"dpi" : 300}
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd


# from cadFace.vis import percentiles_plot
import sci_palettes

try:
    sci_palettes.register_cmap()
except:
    pass
import scienceplots
from pathlib import Path
import pandas as pd


plt.style.use(["nature", "no-latex"])
sns.set_context("paper", font_scale=1.5)
sns.set_palette("nejm")

## 模型训练

### Simple Lasso Regression Build

In [2]:
import pickle

DatasetDir = Path("results/Meta_Prot/dataset/")

Prot_training_df = pd.read_pickle(DatasetDir / "Prot_training_df.pkl")
Meta_training_df = pd.read_pickle(DatasetDir / "Meta_training_df.pkl")
RF_training_df = pd.read_pickle(DatasetDir / "RF_training_df.pkl")
held_out_training_df = pd.read_pickle(DatasetDir / "held_out_training_df.pkl")
held_out_testing_df = pd.read_pickle(DatasetDir / "held_out_testing_df.pkl")

data_cols = pd.read_pickle(DatasetDir / "clean_data_cols.pkl")

In [13]:
training_df_dict = {
    "Prot": Prot_training_df,
    "Meta": Meta_training_df,
    "RF": RF_training_df,
    "PRS": RF_training_df,  # PRS is the same as RF
    "SomtRF": RF_training_df,  # SomtaticRS is the same as RF
}

In [14]:
score_model_dir = Path("results/Meta_Prot/Models/")
Path(score_model_dir).mkdir(parents=True, exist_ok=True)

In [18]:
# train model
from ppp_prediction.model import fit_best_model
from ppp_prediction.model import cal_binary_metrics_bootstrap
import pickle

res = []


score_model_dict = {}

for name, train_imputed in training_df_dict.items():
    X_var = data_cols[name]
    if name not in held_out_testing_df.columns:
        save_obj_dir = f"{score_model_dir}/{name}.pkl"
        if Path(save_obj_dir).exists():
            print(f"Already have {name}, skip")
            model = pickle.load(open(save_obj_dir, "rb"))["model"]
            continue
        (
            model,
            train_metrics,
            test_metrics,
            train_imputed_data,
            held_out_testing_df_data,
            best_models,
        ) = fit_best_model(
            train_df=train_imputed,
            test_df=held_out_testing_df,
            X_var=X_var,
            y_var="incident_cad",
            method_list="Lasso",
            cv=5,
        )

        model_obj = {
            "model": model,
            "train_metrics": train_metrics,
            "test_metrics": test_metrics,
        }
        held_out_testing_df_data.rename(
            columns={"incident_cad_pred": name}, inplace=True
        )
        train_imputed_data.rename(columns={"incident_cad_pred": name}, inplace=True)
        held_out_testing_df[name] = held_out_testing_df_data[name]

        held_out_training_df_data = model.predict(held_out_training_df[X_var])
        held_out_training_df[name] = held_out_training_df_data
        # train_imputed[name] = train_imputed_data[name]
        pickle.dump(model_obj, open(save_obj_dir, "wb"))
    else:
        to_cal_test = held_out_testing_df[["incident_cad", name]].dropna()
        test_metrics = cal_binary_metrics_bootstrap(
            to_cal_test["incident_cad"],
            to_cal_test[name],
            ci_kwargs={"n_resamples": 100},
        )

    test_metrics["method"] = name
    res.append(test_metrics)


res_df = pd.DataFrame(res)
res_df

,AUC,AUC_UCI,AUC_LCI,ACC,ACC_UCI,ACC_LCI,Macro_F1,Macro_F1_UCI,Macro_F1_LCI,Sensitivity,Sensitivity_UCI,Sensitivity_LCI,Specificity,Specificity_UCI,Specificity_LCI,APR,APR_UCI,APR_LCI,N,N_case,N_control,method
0,0.764079,0.782313,0.747169,0.759617,0.768470,0.753393,0.542057,0.553179,0.534301,0.640625,0.672859,0.609501,0.766530,0.774126,0.759816,0.177699,0.197904,0.153631,13986,768.0,13218.0,Prot
1,0.713620,0.728837,0.698144,0.615258,0.621956,0.605350,0.460021,0.467290,0.453472,0.720052,0.747119,0.687607,0.609169,0.617378,0.600936,0.133688,0.149971,0.112269,13986,768.0,13218.0,Meta
2,0.743848,0.765143,0.732164,0.681968,0.690103,0.676241,0.496115,0.503992,0.488453,0.679687,0.717309,0.643859,0.682100,0.690557,0.673980,0.141351,0.155537,0.121771,13986,768.0,13218.0,RF
3,0.632617,0.650640,0.615693,0.641550,0.649810,0.633057,0.459589,0.466148,0.454437,0.558170,0.591954,0.526596,0.646393,0.655737,0.638223,0.088210,0.098701,0.075279,13935,765.0,13170.0,PRS
4,0.574767,0.591105,0.553908,0.561717,0.571169,0.551199,0.416457,0.423376,0.409718,0.562500,0.593976,0.522828,0.561671,0.573148,0.552518,0.069527,0.075502,0.061028,11180,624.0,10556.0,SomtRF


In [20]:
score_save_dir = Path("results/Meta_Prot/score/")
Path(score_save_dir).mkdir(parents=True, exist_ok=True)

held_out_training_df.to_pickle(score_save_dir / "held_out_training_df.pkl")
held_out_testing_df.to_pickle(score_save_dir / "held_out_testing_df.pkl")
res_df.to_csv(Path("results/Meta_Prot") / "basic_model_metrics.csv")

### Bootstrap Feature Selection